In [28]:
import pandas as pd 
import wandb
import numpy as np
api = wandb.Api()

# Project is specified by <entity/project-name>
sweep = api.sweep("app-scripts_analyse/sweeps/h0lq7f5l")
#sweep = api.sweep("app-scripts_analyse_play/sweeps/5refs3e2")

run_list = []
i = 0
for run in sweep.runs: 
    print(f"processing run {i}", end="\r")
    i += 1
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    run_list.append({
        "name": run.name,
        **run.summary._json_dict,
        **{k: v for k,v in run.config.items() if not k.startswith('_')}
    })

runs_df = pd.DataFrame(run_list)

wandb: WARNING A graphql request initiated by the public wandb API timed out (timeout=9 sec). Create a new API with an integer timeout larger than 9, e.g., `api = wandb.Api(timeout=19)` to increase the graphql timeout.


In [29]:
df = runs_df[['t0', 'delta', 'epsilon']]
df.head()

,t0,delta,epsilon
0,0.338433,0.5,1.0
1,0.302548,0.2,0.3
2,0.366076,1.0,0.6
3,0.330497,0.9,0.0
4,0.335287,0.4,0.0


In [30]:
n = 7

In [31]:
group = df.groupby(['delta'])
means = group.mean()
stds = group.std()

top_n = means['t0'].values.argsort()[::-1][:n]

for i in range(n):
    print(f"{i + 1} best delta: {means.index.values[top_n[i]]} "
          f" with mean score {means.values[top_n[i]][0]}+-{stds.values[top_n[i]][0]}",
          f"over {group.size().values[top_n[i]]} runs \n\n ")

1 best delta: 1.0  with mean score 0.34185242528172466+-0.009859540322867448 over 65 runs 

 
2 best delta: 0.9  with mean score 0.3402653195695146+-0.00950760616523524 over 58 runs 

 
3 best delta: 0.8  with mean score 0.33771367910221417+-0.008876478064140169 over 63 runs 

 
4 best delta: 0.7  with mean score 0.33749493383152646+-0.007798533775104093 over 58 runs 

 
5 best delta: 0.6  with mean score 0.33540053895149435+-0.00858223765042975 over 52 runs 

 
6 best delta: 0.5  with mean score 0.33258951489130767+-0.009204437839074755 over 61 runs 

 
7 best delta: 0.4  with mean score 0.3284885582274232+-0.007968131968201446 over 63 runs 

 


In [32]:
group = df.groupby(['epsilon'])
means = group.mean()
stds = group.std()

top_n = means['t0'].values.argsort()[::-1][:n]

for i in range(n):
    print(f"{i + 1} best epsilon: {means.index.values[top_n[i]]} "
          f" with mean score {means.values[top_n[i]][0]}+-{stds.values[top_n[i]][0]}",
          f"over {group.size().values[top_n[i]]} runs \n\n ")

1 best epsilon: 0.9  with mean score 0.3352570371892336+-0.012241452615183229 over 62 runs 

 
2 best epsilon: 0.0  with mean score 0.33444596066825366+-0.007815976584327476 over 55 runs 

 
3 best epsilon: 1.0  with mean score 0.33364199563595254+-0.014050788172877096 over 51 runs 

 
4 best epsilon: 0.4  with mean score 0.3329664043961534+-0.011125531809146003 over 51 runs 

 
5 best epsilon: 0.8  with mean score 0.3325653802922443+-0.014013092154222096 over 68 runs 

 
6 best epsilon: 0.6  with mean score 0.33237149283070544+-0.012093728469009584 over 52 runs 

 
7 best epsilon: 0.5  with mean score 0.3312421540243196+-0.010193262586157215 over 55 runs 

 


In [33]:
group = df.groupby(['epsilon', 'delta'])
means = group.mean()
stds = group.std()

top_n = means['t0'].values.argsort()[::-1][:n]

for i in range(n):
    print(f"\n\n {i + 1} best combination: {list(zip(means.index.names, means.index.values[top_n[i]]))} "
          f" with mean score {means.values[top_n[i]][0]}+-{stds.values[top_n[i]][0]}",
          f"over {group.size().values[top_n[i]]} runs")



 1 best combination: [('epsilon', 0.9), ('delta', 0.9)]  with mean score 0.35070828025477707+-0.003934270700223174 over 5 runs


 2 best combination: [('epsilon', 0.7), ('delta', 1.0)]  with mean score 0.34936305732484074+-0.008980512256517893 over 3 runs


 3 best combination: [('epsilon', 0.6), ('delta', 1.0)]  with mean score 0.34887898089171976+-0.01178877138793313 over 5 runs


 4 best combination: [('epsilon', 0.8), ('delta', 1.0)]  with mean score 0.3475959963603276+-0.009337784440264259 over 7 runs


 5 best combination: [('epsilon', 0.8), ('delta', 0.9)]  with mean score 0.3475745222929937+-0.011412497055871065 over 5 runs


 6 best combination: [('epsilon', 1.0), ('delta', 1.0)]  with mean score 0.34590658174097666+-0.006788383712947875 over 9 runs


 7 best combination: [('epsilon', 1.0), ('delta', 0.9)]  with mean score 0.3457088262056415+-0.0070010457619414 over 7 runs


In [37]:
group = df.groupby(['epsilon', 'delta'])
means = group.mean()
stds = group.std()

top_n = means['t0'].values.argsort()[::-1][:5]
table = """
\\begin{tabular}{l|c|c|c}\n \\toprule
Iterations & $\\delta$ & $\\epsilon$ & $\\text{APAMCTS}_{50}$  \\\\ \n"""

for i in range(5):
    mu = means.values[top_n[i]][0]
    std = stds.values[top_n[i]][0]

    mu = np.around(mu, 3)
    std = np.around(std, 3)

    eps, delta = means.index[top_n[i]]
    
    table += "\\hline $10^2$ & {} & {} & ${:.3f}\\pm{:.3f}$ \\\\ \n".format(delta, eps, mu, std)
    
table += "\\bottomrule\n\end{tabular}"
print(table)


\begin{tabular}{l|c|c|c}
 \toprule
Iterations & $\delta$ & $\epsilon$ & $\text{APAMCTS}_{50}$  \\ 
\hline $10^2$ & 0.9 & 0.9 & $0.351\pm0.004$ \\ 
\hline $10^2$ & 1.0 & 0.7 & $0.349\pm0.009$ \\ 
\hline $10^2$ & 1.0 & 0.6 & $0.349\pm0.012$ \\ 
\hline $10^2$ & 1.0 & 0.8 & $0.348\pm0.009$ \\ 
\hline $10^2$ & 0.9 & 0.8 & $0.348\pm0.011$ \\ 
\bottomrule
\end{tabular}
